In [1]:
import numpy as np
import igl
import meshplot as mp

In [2]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

In [3]:
# mp(V,T)

# Reading point cloud

In [4]:
# pi is point array, v is face array
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 8})

E:\Programs\Anaconoda\envs\py38\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Setting up the Constraints

In [6]:
max_coordinate = np.max(pi, axis = 0)
min_coordinate = np.min(pi, axis = 0)
print("max_coordinate: "+ str(max_coordinate))
print("min_coordinate: "+ str(min_coordinate))

v_num = pi.shape[0]
print("v_num", v_num)

max_coordinate: [32.7 16.7  5.8]
min_coordinate: [-22.7 -63.7 -99. ]
v_num 366


In [7]:
# return id of closed point
def find_closed_point(point, points):
    # distances of each point
    distances = np.linalg.norm(points - point, axis = 1)
    return np.argmin(distances)

In [10]:
# get appropriate eps for this point
def update_eps(_eps, _x_id, _x_coord, _norm):
    x_eps = _x_coord + _norm * _eps
    if(find_closed_point(x_eps, pi) == _x_id):
        return _eps
    else:
        return update_eps(_eps/2, _x_id, _x_coord, _norm)
    

In [12]:
# initial eps
eps = 0.01 * igl.bounding_box_diagonal(pi)
# print(eps)

# get v+ and v- for each vertex v
constraint_v_coord_arr = np.zeros((3*v_num, 3), dtype = np.float64)
constraint_fval_arr = np.zeros(3*v_num, dtype = np.float64)
constraint_color_arr = np.zeros((3*v_num, 3), dtype = np.float64) #RGB
for v_id in range(v_num):
    coord = pi[v_id]
    norm = ni[v_id]
    constraint_v_coord_arr[3*v_id] = coord
    constraint_fval_arr[3*v_id] = 0.0
    constraint_color_arr[3*v_id] = [0.0, 0.0, 1.0]
    # for v+
    eps = update_eps(eps, v_id, coord, norm)
    constraint_v_coord_arr[3*v_id + 1] = coord + eps * norm
    constraint_fval_arr[3*v_id + 1] = eps
    constraint_color_arr[3*v_id + 1] = [1.0, 0.0, 0.0]
    # for v-
    eps = update_eps(eps, v_id, coord, -norm)
    constraint_v_coord_arr[3*v_id + 2] = coord - eps * norm
    constraint_fval_arr[3*v_id + 2] = -eps
    constraint_color_arr[3*v_id + 2] = [0.0, 1.0, 0.0]

In [14]:
mp.plot(constraint_v_coord_arr, c = constraint_color_arr, shading={"point_size": 8})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0120944…

# MLS function

In [9]:
# Parameters
bbox_min = np.array([-1., -1., -1.])
bbox_max = np.array([1., 1., 1.])
bbox_diag = np.linalg.norm(bbox_max - bbox_min)
# print(bbox_diag)
n = 10

3.4641016151377544


In [11]:
# Generate grid n x n x n

x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

#Compute implicit sphere function
center = np.array([0., 0., 0.])
radius = 1
fx = np.linalg.norm(x-center, axis=1) - radius

In [12]:
# Treshold fx to visualize inside outside

ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1
mp.plot(x, c=ind, shading={"point_size": 0.1,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Marching to extract surface

In [ ]:
# Marcing tet to extract surface

sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
mp.plot(sv, sf, shading={"wireframe": True})